# Kolonnensimulation mit variablen echten Anfangsbedingungen

In [1]:
import casadi
import numpy as np
import pandas as pd
import os, sys
import pickle
import matplotlib as mpl
from matplotlib import pyplot as plt

KeyboardInterrupt: 

In [ ]:
mpl.rcParams["font.size"] = 9
mpl.rcParams["lines.linewidth"] = 3
mpl.rcParams["axes.grid"] = True

In [ ]:
from column_variable_init_conds import template_model, template_simulator, update_u

In [ ]:
index = 1000
model = template_model(init_ind=index)
simulator = template_simulator(model,init_ind=index)

params_simulator = {
    "integration_tool": "idas",
    "abstol": 1e-3,
    "reltol": 1e-3,
    "t_step": 0.1,
}
simulator.set_param(**params_simulator)
simulator.setup()
simulator.set_initial_guess()   

In [ ]:
u0 = simulator.u0.master
t0 = index * params_simulator["t_step"]

for step in range(100001):
    process_time = simulator.data["_time"][-1][0] if step > 0 else 0
    update_u(simulator,trajectory=-1,use_time=True,time=process_time+t0,getoutput=True)
    simulator.make_step(u0)
    if step%200 == 0:
        print(step)
    #if step>0:
        #if simulator.data["_time"][-1][0]+t0 > 10:
            

In [ ]:
    timestep = params_simulator["t_step"]
    fig, ax = plt.subplots(2,sharex=True)
    ax[0].plot(simulator.data["_time"],
            simulator.data["_z","e0_V_V_st9"],label="st 9")
    ax[0].plot(simulator.data["_time"],
            simulator.data["_z","e0_V_V_st7"],label="st 7")
    ax[0].plot(simulator.data["_time"],
            simulator.data["_z","e0_V_V_st5"],label="st 4")
    ax[0].plot(simulator.data["_time"],
            simulator.data["_z","e0_V_V_st3"],label="st 2")
    ax[0].set_ylabel("Dampfvolumen [?]")
    ax[0].legend()
    ax[0].set_title("V_V_i at stages 2,4,7 and 9")
    
    ax[1].plot(simulator.data["_time"],simulator.data["_z","e0_V_L_st9"],label="")
    ax[1].set_ylabel("liquid volume")
    ax[1].set_xlabel("Time in seconds")
    ax[1].set_title("V_liquid at reboiler")
    
    
    fig, ax1 = plt.subplots(2,sharex=True)
    ax1[0].plot(simulator.data["_time"],
                simulator.data["_z","e0_y_st9_i1"],label="y_1")
    ax1[0].plot(simulator.data["_time"],
                simulator.data["_z","e0_y_st9_i2"],label="y_2")
    ax1[0].plot(simulator.data["_time"],
                simulator.data["_z","e0_y_st9_i3"],label="y_N2")
    ax1[0].set_ylabel("vapor fraction")
    ax1[0].legend()
    ax1[0].set_title("x_i and y_i at reboiler")
    
    
    
    ax1[1].plot(simulator.data["_time"],
                simulator.data["_z","e0_x_st9_i1"],label="x_1")
    ax1[1].plot(simulator.data["_time"],
                simulator.data["_z","e0_x_st9_i2"],label="x_2")
    ax1[1].plot(simulator.data["_time"],
                simulator.data["_z","e0_x_st9_i3"],label="x_N2")
    ax1[1].set_ylabel("liquid fraction")
    ax1[1].set_xlabel("Time in seconds")
    ax1[1].legend()
    plt.show()

In [ ]:
def compare_plots(simulator,trajectory,t0,var_dict:dict,index=0):
    i = 100
    
    for var_type, variables in var_dict.items():
        if var_type.lower() == "z":
            spec = "_z"
            color = "b"
        elif var_type.lower() == "u":
            spec = "_u"
            color = "k"
        elif var_type.lower() == "x":
            spec = "_x"
            color = "b"
        
        for z_var in variables:
            ende= 30000
            i +=1
            plt.figure(num=i)
            plt.plot(simulator.data["_time"]+t0,simulator.data[spec,z_var],"{}o-".format(color),label="model")
            plt.plot(trajectory["Time"],trajectory["Flowsheet."+z_var],"y",label="data")
            plt.legend()
            plt.xlabel("Time in seconds")
            plt.title(z_var)
            plt.xlim([0,400])

trajectory = pd.read_pickle("./Data/alldata.pck")
var_dict = {"x":[],"z":["e0_T_st9","e0_V_L_st9"],"u":["e0_F_F_st9","e0_greek_sigma_R"]}
compare_plots(simulator,trajectory,t0,var_dict,index=index)

In [ ]:
len(simulator.data["_x"])